In [ ]:
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta

In [ ]:
class FixedIncomeAsset:
    def __init__(self, face_value, coupon_rate, price, maturity_years, frequency=2, issue_date=None):
        """
        Initializes a FixedIncomeAsset object.

        Parameters:
            face_value (float): The bond's face value.
            coupon_rate (float): Annual coupon rate as a decimal.
            price (float): Current market price of the bond.
            maturity_years (float): Years until maturity.
            frequency (int): Coupon payment frequency per year (default is 2 for semi-annual).
            issue_date (datetime): The bond's issue date.
        """
        self.face_value = face_value
        self.coupon_rate = coupon_rate
        self.price = price
        self.maturity_years = maturity_years
        self.frequency = frequency
        self.issue_date = issue_date if issue_date else datetime.today()
        self.maturity_date = self.issue_date + timedelta(days=maturity_years * 365)
    
    def cash_flows(self):
        """
        Calculates the cash flows of the bond.

        Returns:
            np.array: Cash flows array for each period until maturity.
        """
        periods = int(self.maturity_years * self.frequency)
        coupon_payment = self.face_value * self.coupon_rate / self.frequency
        cash_flows = np.full(periods, coupon_payment)
        cash_flows[-1] += self.face_value  # Adding face value at maturity
        return cash_flows

    def yield_to_maturity(self):
        """
        Calculates the bond's yield to maturity (YTM) using numerical optimization.

        Returns:
            float: Yield to maturity as a decimal.
        """
        cash_flows = self.cash_flows()
        periods = len(cash_flows)

        def ytm_func(ytm):
            discount_factors = (1 + ytm / self.frequency) ** np.arange(1, periods + 1)
            return np.sum(cash_flows / discount_factors) - self.price

        ytm = minimize(ytm_func, 0.05).x[0]
        return ytm * self.frequency

    def duration(self):
        """
        Calculates the Macaulay duration of the bond.

        Returns:
            float: Macaulay duration.
        """
        ytm = self.yield_to_maturity()
        cash_flows = self.cash_flows()
        periods = len(cash_flows)
        discount_factors = (1 + ytm / self.frequency) ** np.arange(1, periods + 1)
        present_values = cash_flows / discount_factors
        weighted_maturities = np.arange(1, periods + 1) * present_values
        macaulay_duration = np.sum(weighted_maturities) / np.sum(present_values)
        return macaulay_duration / self.frequency

    def modified_duration(self):
        """
        Calculates the bond's modified duration.

        Returns:
            float: Modified duration.
        """
        macaulay_duration = self.duration()
        ytm = self.yield_to_maturity()
        return macaulay_duration / (1 + ytm / self.frequency)

    def convexity(self):
        """
        Calculates the convexity of the bond.

        Returns:
            float: Convexity.
        """
        ytm = self.yield_to_maturity()
        cash_flows = self.cash_flows()
        periods = len(cash_flows)
        discount_factors = (1 + ytm / self.frequency) ** (np.arange(1, periods + 1) + 2)
        present_values = cash_flows / discount_factors
        convexity_sum = np.sum(present_values * np.arange(1, periods + 1) * (np.arange(1, periods + 1) + 1))
        convexity = convexity_sum / (self.price * (1 + ytm / self.frequency) ** 2)
        return convexity

    def value_at_risk(self, confidence_level=0.95):
        """
        Estimates the value at risk (VaR) of the bond based on duration and convexity.

        Parameters:
            confidence_level (float): The confidence level for VaR calculation.

        Returns:
            float: Estimated VaR for the bond.
        """
        modified_duration = self.modified_duration()
        convexity = self.convexity()
        yield_shock = np.percentile(np.random.normal(0, 0.01, 10000), 1 - confidence_level)
        price_change = -self.price * (modified_duration * yield_shock + 0.5 * convexity * yield_shock**2)
        return price_change

    def election_cycle_risk_analysis(self, election_date):
        """
        Analyzes the bond's performance over an election cycle, six months before and 
        two weeks post-election.

        Parameters:
            election_date (datetime): The date of the election.

        Returns:
            dict: A dictionary with metrics calculated for the election cycle.
        """
        start_period = election_date - timedelta(days=6*30)
        end_period = election_date + timedelta(days=14)
        
        if self.issue_date > end_period or self.maturity_date < start_period:
            return {"error": "Bond is not active during the specified election cycle"}

        ytm = self.yield_to_maturity()
        mod_duration = self.modified_duration()
        convexity = self.convexity()
        election_vasr = self.value_at_risk()
        
        analysis_results = {
            "yield_to_maturity": ytm,
            "modified_duration": mod_duration,
            "convexity": convexity,
            "value_at_risk": election_vasr,
            "start_period": start_period,
            "end_period": end_period
        }

        return analysis_results
